In [ ]:
!pip install sentence-transformers langchain_huggingface langchain_pinecone pinecone-client

In [ ]:
from sentence_transformers import CrossEncoder
from pinecone import Pinecone
from langchain_huggingface import HuggingFaceEmbeddings

import os
os.environ['PINECONE_API_KEY'] = <PINECONE_API_KEY>
pinecone_api_key = os.getenv("PINECONE_API_KEY")

pc= Pinecone(api_key= pinecone_api_key)
index = pc.Index("profilestore")

embedder = HuggingFaceEmbeddings(model_name = 'BAAI/bge-base-en-v1.5')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

folder_path = "/content/drive/My Drive/GenAI/"

Mounted at /content/drive


In [ ]:
# ============  Generate Test Data ==================

import random
import pandas as pd

# Load Excel file
df1 = pd.read_excel(folder_path + "testdata1.xlsx")
df2 = pd.read_excel(folder_path + "testdata2.xlsx")

df= pd.concat([df1, df2], ignore_index=True)

# Create a copy of answers and shuffle them
shuffled_answers = df["Answer"].sample(frac=1, random_state=42).reset_index(drop=True)

# Assign relevance scores: 1.0 for correct pairs, 0.0 for incorrect (shuffled) pairs
positive_samples = list(zip(df["Question"], df["Answer"], [1.0] * len(df)))
negative_samples = list(zip(df["Question"], shuffled_answers, [0.0] * len(df)))

# Combine positive & negative samples
train_data = positive_samples + negative_samples

# Shuffle training data
random.shuffle(train_data)

# Store in excel
df_train = pd.DataFrame(train_data, columns=["Question", "Answer", "Relevance"])
df_train.to_excel(folder_path + "train_data.xlsx", index=False)

In [ ]:
from torch.utils.data import DataLoader
from sentence_transformers import InputExample

# Prepare data for training
train_samples = [InputExample(texts=[str(q), str(a)], label=score) for q, a, score in train_data]

model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# Convert to DataLoader (batch_size set here)
train_dataloader = DataLoader(train_samples, batch_size=8, shuffle=True)

# Train the model
model.fit(
    train_dataloader=train_dataloader,
    epochs=3, # CHANGE LATER
    warmup_steps=100
)

model.save(folder_path + "CrossEncoderModel")

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1352 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1352 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1352 [00:00<?, ?it/s]

In [ ]:
# Step 1: Retrieve Top-50 Results from Pinecone (Vector Search)
query = "What does 'Schedule A' determine?"

query_vector = embedder.embed_query(query)
pinecone_results = index.query(vector=query_vector, top_k=500, include_metadata=True)
print(pinecone_results["matches"])

# Step 2: Use a Cross-Encoder for Reranking
cross_encoder = model

# Prepare query-chunk pairs
query_chunk_pairs = [(query, "".join(doc["metadata"]["text"])) for doc in pinecone_results["matches"]]
print(query_chunk_pairs)

cross_encoder_scores = cross_encoder.predict(query_chunk_pairs)

# Step 3: Sort by Cross-Encoder Scores
reranked_results = sorted(
    zip(pinecone_results["matches"], cross_encoder_scores),
    key=lambda x: x[1], reverse=True
)[:10]  # Take top 10

# Output Final Ranked Results
for res, score in reranked_results:
    print(f"Chunk: {res['metadata']['text']}, Score: {score}")